In [1]:
import numpy as np
import pandas as pd
import os
import torch

os.chdir("../")

from src.conf import LAYERS_DIMS, MODEL_FEATURES
from src.utils import get_device
from src.data import DataLoaders
from src.models import FraudAutoEncoder

## Data Gen

In [2]:
n_, k_ = 1_000, 300

In [3]:
pd.DataFrame(
    data=np.random.uniform(** {"low": -1, "high": 1, "size": [n_, k_]}),
    columns=[f"X_{str(i).zfill(3)}" for i in range(1, 1+k_)]
).to_parquet("./data/simulated_raw_data_new_arrival.gzip", compression="gzip")

In [4]:
X = pd.read_parquet("/home/onyxia/work/fraud_detection/data/simulated_raw_data_new_arrival.gzip")

## Model Loading

In [5]:
for _, (x, y) in enumerate(DataLoaders(X, [0.7, 0.2, 0.1]).get_dataloaders()[0]):
    assert y.shape[0] == x.shape[0]

In [14]:
encode_decode_errors = pd.DataFrame(
    columns=["id", "encode_decode_error", "split"]
)

encode_decode_errors = pd.concat([
    encode_decode_errors,
    pd.DataFrame(data=[{
    "id": 1,
    "encode_decode_error": 2,
    "split": "train"}])
])

encode_decode_errors

,id,encode_decode_error,split
0,1,2,train


In [ ]:
DEVICE = get_device(1)

_LAYERS_DIMS = LAYERS_DIMS(
    INPUT_DIM=300,
    HIDDEN_DIM=150,
    CODE_DIM=35
)
model = FraudAutoEncoder(_LAYERS_DIMS).to(DEVICE)
model_hyperparams = MODEL_FEATURES(
    LEARNING_RATE=1e-3,
    N_EPOCHS=100
)
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=model_hyperparams.LEARNING_RATE
)

loss_criterion = torch.nn.MSELoss()

In [ ]:
checkpoint = torch.load("./models/best_model_simulated_data.ckpt", map_location="cuda")
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']

model.eval()

In [ ]:
error = torch.nn.MSELoss()

_x = torch.from_numpy(X[0]).to(DEVICE).to(torch.float32)

with torch.no_grad():
    loss = torch.nn.MSELoss()
    print(error(_x, model(_x)))